This notebook that does a little wrangling of the gtfs-validator results we have stored into DataFrames. There are 4 main DFs here:

* `status` - one row per agency x url
* `tidy_gtfs_files` - for successfully unzipped gtfs data, one row per file (including validation.json)
* `tidy_notice_codes` - one row per "code" in the validator results. A code is a type of violation. E.g. "invalid_phone_number".
* `tidy_notice_details` - one row per instance violating the code (e.g. each invalid phone number).

In [1]:
import gcsfs
import pandas as pd
import json

from siuba.dply.vector import row_number
from siuba import *

fs = gcsfs.GCSFileSystem(project="cal-itp-data-infra")

BUCKET_URL = "gs://gtfs-data/schedule/2021-04-05T00:00:00+00:00"
DATA_URL_TMPL = BUCKET_URL + "/{itp_id}/{url_number}"

In [2]:
status = pd.read_csv(BUCKET_URL + "/status.csv") >> select(-_.startswith("Unnamed"))

status_success = status >> filter(_.status == "success")

# Note that I've opened an issue in siuba to implement rowwise(),
# to replace some cumbersome parts of this group_by -> mutate
# could also use df.apply(lambda x: ..., axis = 1)
tidy_gtfs_files = (status_success
    >> group_by(tmp = row_number(_))
    >> mutate(
        gtfs_url = lambda d: DATA_URL_TMPL.format(**d.squeeze()),
        gtfs_files = lambda d: [fs.listdir(d.squeeze()["gtfs_url"])]
    )
    >> ungroup()
    >> pipe(_.explode("gtfs_files"))
    >> mutate(gtfs_file_name = _.gtfs_files.apply(lambda x: x['name']))
)

In [3]:
validation_files = (tidy_gtfs_files
  >> filter(_.gtfs_file_name.str.contains("validation\\.json"))
  >> group_by(tmp = row_number(_))
  >> mutate(
      validation = lambda d: [json.load(fs.open(d.squeeze().gtfs_file_name))],
      notices = lambda d: [d["validation"].iloc[0]["data"]["report"]["notices"]],
      n_codes = lambda d: len(d["notices"].iloc[0])
  )
  >> ungroup()
)

### Sanity check that each successful unzip has validation.json

In [4]:
validation_files >> count()

,n
0,171


In [5]:
status_success >> count()

,n
0,171


In [6]:
# example validation.json
# note that the reporting across codes doesn't look too consistent. For example,
# one code reports the file where the error occurs, but another doesn't.
# maybe some codes don't map to a single file, or just a lack of consistency?
validation_files.validation[0]

{'version': 'v2.0.0',
 'data': {'report': {'notices': [{'code': 'duplicate_fare_rule_zone_id_fields',
     'severity': 'ERROR',
     'totalNotices': 8,
     'notices': [{'csvRowNumber': 61,
       'fareId': 'L17_E',
       'previousCsvRowNumber': 59,
       'previousFareId': 'TL25'},
      {'csvRowNumber': 62,
       'fareId': 'L17_W',
       'previousCsvRowNumber': 60,
       'previousFareId': 'TL25'},
      {'csvRowNumber': 79,
       'fareId': 'L17_E',
       'previousCsvRowNumber': 77,
       'previousFareId': 'TL25'},
      {'csvRowNumber': 80,
       'fareId': 'L17_W',
       'previousCsvRowNumber': 78,
       'previousFareId': 'TL25'},
      {'csvRowNumber': 87,
       'fareId': 'L17_E',
       'previousCsvRowNumber': 85,
       'previousFareId': 'TL25'},
      {'csvRowNumber': 88,
       'fareId': 'L17_W',
       'previousCsvRowNumber': 86,
       'previousFareId': 'TL25'},
      {'csvRowNumber': 94,
       'fareId': 'L17_E',
       'previousCsvRowNumber': 92,
       'previousF

## Tidy notices

Note that validation.json has two levels of "notices":

* codes - the violated rule
* details - each specific case that violated the rule

Both are called "notices" in the json data. E.g...

```python
{
  data:
    report:
      # note the code, versus nested notices data
      notices: [{code: "some_code", notices: [...]}]
      
    # hopefully should be empty
    system_errors: {...}
}
```

### Notice codes

In [7]:
notice_codes = (validation_files
           .assign(notices = lambda d: d["notices"].transform(pd.DataFrame))
          )

# note that siuba unnest currently requires resetting index
tidy_notice_codes = (notice_codes.reset_index(drop=True)
  >> select(_.agency_name, _.itp_id, _.url_number, _.notices)
  >> unnest("notices")
  )

In [8]:
tidy_notice_codes.head()

,agency_name,itp_id,url_number,code,severity,totalNotices,notices
0,AC Transit,4,0,duplicate_fare_rule_zone_id_fields,ERROR,8,"[{'csvRowNumber': 61, 'fareId': 'L17_E', 'prev..."
1,AC Transit,4,0,invalid_phone_number,ERROR,1,"[{'filename': 'agency.txt', 'csvRowNumber': 2,..."
2,Alhambra Community Transit,6,0,invalid_phone_number,ERROR,1,"[{'filename': 'agency.txt', 'csvRowNumber': 2,..."
3,Alhambra Community Transit,6,0,unknown_column,INFO,29,"[{'filename': 'stop_times.txt', 'fieldName': '..."
4,Alhambra Community Transit,6,0,unknown_file,INFO,13,"[{'filename': 'calendar_attributes.txt'}, {'fi..."


In [9]:
# most common codes
tidy_notice_codes >> count(_.code, sort = True)

,code,n
0,invalid_phone_number,152
1,unknown_column,130
2,unknown_file,129
3,duplicate_route_name,32
4,decreasing_or_equal_shape_distance,27
5,stop_time_timepoint_without_times,24
6,feed_expiration_date,22
7,duplicate_fare_rule_zone_id_fields,20
8,route_color_contrast,15
9,leading_or_trailing_whitespaces,13


### Notice details

In [10]:
# create a giant table of notices
# note that it's the "notices" list of dicts holding each code violation
# unpacked into separate columns. many are NA because notice details can have 
# different fields, but the big table helps wade through all the different forms
# they can take.
notice_codes_long = tidy_notice_codes.explode("notices").reset_index(drop = True)

tidy_notice_details = notice_codes_long.join(
        pd.DataFrame(notice_codes_long.notices.tolist())
)


In [11]:
tidy_notice_details.head()

,agency_name,itp_id,url_number,code,severity,totalNotices,notices,csvRowNumber,fareId,previousCsvRowNumber,...,routeLongName,prevStopTimeDistTraveled,prevStopSequence,routeDesc,speedkmh,firstStopSequence,lastStopSequence,stopId,stopName,stopShapeThresholdMeters
0,AC Transit,4,0,duplicate_fare_rule_zone_id_fields,ERROR,8,"{'csvRowNumber': 61, 'fareId': 'L17_E', 'previ...",61.0,L17_E,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AC Transit,4,0,duplicate_fare_rule_zone_id_fields,ERROR,8,"{'csvRowNumber': 62, 'fareId': 'L17_W', 'previ...",62.0,L17_W,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AC Transit,4,0,duplicate_fare_rule_zone_id_fields,ERROR,8,"{'csvRowNumber': 79, 'fareId': 'L17_E', 'previ...",79.0,L17_E,77.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AC Transit,4,0,duplicate_fare_rule_zone_id_fields,ERROR,8,"{'csvRowNumber': 80, 'fareId': 'L17_W', 'previ...",80.0,L17_W,78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AC Transit,4,0,duplicate_fare_rule_zone_id_fields,ERROR,8,"{'csvRowNumber': 87, 'fareId': 'L17_E', 'previ...",87.0,L17_E,85.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
(tidy_notice_details
  >> filter(_.itp_id == 386, _.code == "stop_time_timepoint_without_times")
  >> select(_[0:5], _.csvRowNumber, _.tripId, _.stopSequence, _.specifiedField)
  >> head()
  )

,agency_name,itp_id,url_number,code,severity,csvRowNumber,tripId,stopSequence,specifiedField
415763,Yuma County Area Transit,386,0,stop_time_timepoint_without_times,WARNING,3.0,352,49.0,arrival_time
415764,Yuma County Area Transit,386,0,stop_time_timepoint_without_times,WARNING,3.0,352,49.0,departure_time
415765,Yuma County Area Transit,386,0,stop_time_timepoint_without_times,WARNING,4.0,352,48.0,arrival_time
415766,Yuma County Area Transit,386,0,stop_time_timepoint_without_times,WARNING,4.0,352,48.0,departure_time
415767,Yuma County Area Transit,386,0,stop_time_timepoint_without_times,WARNING,5.0,352,47.0,arrival_time
